In [92]:
import tensorflow as tf

In [86]:
import transformers

In [87]:
from transformers import BertTokenizer,TFBertForPreTraining

In [88]:
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model=TFBertForPreTraining.from_pretrained("bert-base-uncased")

All PyTorch model weights were used when initializing TFBertForPreTraining.

All the weights of TFBertForPreTraining were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


In [89]:
model.summary() 

Model: "tf_bert_for_pre_training_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 nsp___cls (TFBertNSPHead)   multiple                  1538      
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  24459834  
                                                                 
Total params: 110106428 (420.02 MB)
Trainable params: 110106428 (420.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
with open("clean.txt",'r') as fp:
    text=fp.read().split("\n")

In [7]:
len(text)

507

In [8]:
text[:5]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander."]

# NSP

In [9]:
bag=[sentence for paragraph in text for sentence in paragraph.split('.') if sentence!='']

In [10]:
len(bag)

1372

In [11]:
import random

sentence_a=[]
sentence_b=[]
label=[]

for paragraph in text:
    sentences=[
        sentence for sentence in paragraph.split('.') if sentence!=''
    ]
    num_sentences=len(sentences)
    if num_sentences>1:
        start=random.randint(0,num_sentences-2)
        sentence_a.append(sentences[start])
        if random.random()>0.5:
            sentence_b.append(sentences[start+1])
            label.append(0)
            
        else:
            sentence_b.append(bag[random.randint(0,len(bag)-1)])
            label.append(1)

In [12]:
len(sentence_a),len(sentence_b),len(label)

(317, 317, 317)

In [13]:
sentence__a=sentence_a[:15]
sentence__b=sentence_b[:15]
label_=label[:15]

In [14]:
len(sentence__a),len(sentence__b),len(label_)

(15, 15, 15)

In [15]:
inputs=tokenizer(sentence__a,sentence__b,padding=True,truncation=True,return_tensors='tf')

In [16]:
inputs.input_ids.shape

TensorShape([15, 181])

In [17]:
inputs["next_sentence_label"]=tf.transpose(tf.convert_to_tensor([label_]),(1,0))

In [18]:
inputs.next_sentence_label.shape

TensorShape([15, 1])

# MLM

In [27]:
inputs["labels"]=tf.identity(inputs.input_ids)

In [28]:
inputs.labels.shape

TensorShape([15, 181])

In [29]:
random=tf.random.uniform(inputs.input_ids.shape)

In [30]:
random.shape

TensorShape([15, 181])

In [31]:
array=(inputs.input_ids!=101)

In [32]:
array

<tf.Tensor: shape=(15, 181), dtype=bool, numpy=
array([[False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       ...,
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True]])>

In [33]:
tf.cast(array,dtype=tf.int32)

<tf.Tensor: shape=(15, 181), dtype=int32, numpy=
array([[0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1]])>

In [34]:
tf.where(array,1,0)

<tf.Tensor: shape=(15, 181), dtype=int32, numpy=
array([[0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1]])>

In [35]:
ann=tf.cast((inputs.input_ids!=101),tf.int32)*tf.cast((inputs.input_ids!=102),tf.int32)*tf.cast((inputs.input_ids!=0),tf.int32)
ann

<tf.Tensor: shape=(15, 181), dtype=int32, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])>

In [36]:
mask_arr=tf.cast((random<0.15),tf.int32)*ann

In [37]:
mask_arr

<tf.Tensor: shape=(15, 181), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>

In [38]:
mask_arr=tf.cast(mask_arr,tf.bool)

In [39]:
mask_arr

<tf.Tensor: shape=(15, 181), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

# Numpy

In [40]:
import numpy as np

In [41]:
inputs_ids=np.array(inputs.input_ids)

In [42]:
inputs_ids.shape

(15, 181)

In [101]:
select=[]
for i in range(len(mask_arr)):
    selected=np.where(mask_arr[i]!=False)
    select.append(selected)
    value=103
    for j in selected:
        inputs_ids[i,j]=value

In [44]:
inputs_ids

array([[  101,  2002,  2001, ...,     0,     0,     0],
       [  101,  2057,  2113, ...,     0,     0,     0],
       [  101,  2582,  1010, ...,     0,     0,     0],
       ...,
       [  101,  2129,  2855, ...,  1037,  2775,   102],
       [  101,  1998,  7065, ...,     0,     0,     0],
       [  101,  2295, 15223, ...,     0,     0,     0]])

# Tensorflow

In [45]:
inputs_ids=np.random.random(inputs.input_ids.shape)

In [46]:
for i in range(len(mask_arr)):
    selected=tf.cast(tf.where(mask_arr[i]!=False),dtype=tf.int32)
    value=tf.constant([103],dtype=tf.int32)
    value=tf.broadcast_to(value, [len(selected)])
    inputs_ids[i]=tf.tensor_scatter_nd_update(inputs.input_ids[i], selected,value)

In [47]:
input_ids=tf.cast(tf.convert_to_tensor(inputs_ids),tf.int32)

In [48]:
inputs["input_ids"]=input_ids

# Checking wheather numpy and tensorflow give same output or not

In [49]:
inputs.input_ids==inputs_ids

<tf.Tensor: shape=(15, 181), dtype=bool, numpy=
array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])>

In [50]:
are_equal = np.array_equal(inputs_ids, inputs.input_ids)

In [51]:
are_equal

True

# Inbuilt Training

In [52]:
output=model(**inputs)

In [83]:
output.keys()

odict_keys(['loss', 'prediction_logits', 'seq_relationship_logits'])

In [51]:
output.loss.numpy()

array([61.209457], dtype=float32)

# Custom training (Pretrain)

In [57]:
data_loader = tf.data.Dataset.from_tensor_slices(inputs).batch(5)

In [58]:
data_loader

<_BatchDataset element_spec={'input_ids': TensorSpec(shape=(None, 181), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 181), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 181), dtype=tf.int32, name=None), 'next_sentence_label': TensorSpec(shape=(None, 1), dtype=tf.int32, name=None), 'labels': TensorSpec(shape=(None, 181), dtype=tf.int32, name=None)}>

In [62]:
for i in data_loader:
    print(len(i))

5
5
5


In [90]:
from tensorflow.keras.optimizers import AdamW
optimizer = AdamW(learning_rate=5e-5)

In [91]:
from tqdm import tqdm

In [93]:
epochs = 2

for epoch in range(epochs):
    step = tqdm(data_loader, leave=True)
    
    for batch in step:
        input_ids=batch["input_ids"]
        attention_mask=batch["attention_mask"]
        next_sentence_label=batch["next_sentence_label"]
        token_type_ids=batch["token_type_ids"]
        labels=batch["labels"]
        
        with tf.GradientTape() as tape:
            outputs = model(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                 next_sentence_label=next_sentence_label,
                labels=labels
            )
            loss = outputs.loss
            print(loss)
            
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        step.set_description(f"Epoch {epoch}")
        step.set_postfix(loss=loss.numpy())


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

tf.Tensor([27.046858], shape=(1,), dtype=float32)


Epoch 0:  33%|███████████████████                                      | 1/3 [01:36<03:12, 96.36s/it, loss=[27.046858]]

tf.Tensor([22.904367], shape=(1,), dtype=float32)


Epoch 0:  67%|██████████████████████████████████████                   | 2/3 [02:45<01:20, 80.53s/it, loss=[22.904367]]

...

Epoch 1:  33%|███████████████████▎                                      | 1/3 [00:25<00:51, 25.75s/it, loss=[8.596188]]

tf.Tensor([11.860811], shape=(1,), dtype=float32)


Epoch 1:  67%|██████████████████████████████████████                   | 2/3 [00:46<00:22, 22.84s/it, loss=[11.860811]]

tf.Tensor([6.659443], shape=(1,), dtype=float32)


Epoch 1: 100%|██████████████████████████████████████████████████████████| 3/3 [01:07<00:00, 22.40s/it, loss=[6.659443]]


In [95]:
opp=model(**inputs)

In [96]:
opp.keys()

odict_keys(['loss', 'prediction_logits', 'seq_relationship_logits'])

In [97]:
opp.loss

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([7.51573], dtype=float32)>

In [98]:
opp.prediction_logits.shape

TensorShape([15, 181, 30522])

In [99]:
opp.seq_relationship_logits.shape

TensorShape([15, 2])

# NSP output

In [100]:
for i in range(len(opp.seq_relationship_logits)):
    print("actual:",inputs.next_sentence_label[i].numpy())
    print("predicted:",tf.argmax(opp.seq_relationship_logits[i]).numpy())
    print("\n")

actual: [1]
predicted: 1


actual: [1]
predicted: 1


actual: [1]
predicted: 1


actual: [0]
predicted: 0


actual: [0]
predicted: 0


actual: [0]
predicted: 0


actual: [1]
predicted: 1


actual: [1]
predicted: 1


actual: [0]
predicted: 0


actual: [0]
predicted: 0


actual: [1]
predicted: 1


actual: [0]
predicted: 0


actual: [0]
predicted: 0


actual: [1]
predicted: 1


actual: [0]
predicted: 0




# MLM outputs 


In [114]:
select[7]

(array([  8,  17,  25,  29,  34,  38,  41,  45,  53,  62,  63,  66,  70,
         76,  83,  89, 111, 115], dtype=int64),)

In [103]:
inputs.input_ids.shape

TensorShape([15, 181])

In [104]:
opp.prediction_logits.shape

TensorShape([15, 181, 30522])

In [115]:
tokenizer.decode(inputs.input_ids[7,70])

'[ M A S K ]'

In [116]:
tokenizer.decode(inputs.labels[7,70])

'e v e n'

In [107]:
score=tf.nn.softmax(opp.prediction_logits,axis=-1)

In [108]:
pred=tf.argmax(score,axis=-1)

In [109]:
pred.shape

TensorShape([15, 181])

In [117]:
tokenizer.decode(pred[7,70])

'e v e n'